# Required functions

In [2]:
from samples.corpus_analysis import *

# PLOS/NLM article type mapping

In [2]:
# For mapping the JATS article type onto the PLOS article type, while taking NLM DTD into account.
article_types_map = get_article_types_map()
PLOS_article_types_structured = counter(article_types_map).most_common()
print(PLOS_article_types_structured)

NameError: name 'i' is not defined

In [ ]:
# create .csv file mapping JATS to PLOS article types
article_types_map_to_csv(article_types_map)

# Retracted and corrected articles

## Get list of retracted articles

In [4]:
# article_list = [doi_to_file(doi) for doi in get_random_list_of_dois(count=5000)]
retractions_doi_list, retracted_doi_list = get_retracted_doi_list(article_list=article_list)

1 retracted articles found.


In [65]:
retractions_doi_list

['10.1371/journal.pbio.1002215']

In [3]:
article_list = [doi_to_file('10.1371/journal.pbio.1002215')]

In [61]:
def get_retracted_doi_list(article_list=None, directory=corpusdir):
    """
    Scans through articles in a directory to see if they are retraction notifications,
    scans articles that are that type to find DOIs of retracted articles
    :return: tuple of lists of DOIs for retractions articles, and retracted articles
    """
    retractions_doi_list = []
    retracted_doi_list = []
    if article_list is None:
        article_list = listdir_nohidden(directory)
    for article_file in article_list:
        if check_if_retraction_article(article_file):
            retractions_doi_list.append(file_to_doi(article_file))
            # Look in those articles to find actual articles that are retracted
            retracted_doi = get_related_retraction_article(article_file)[0]
            retracted_doi_list.append(retracted_doi)
            # check linked DOI for accuracy
            if make_regex_bool(full_doi_regex_match.search(retracted_doi)) is False:
                print("{} has incorrect linked DOI field: '{}'".format(article_file, retracted_doi))
    if len(retractions_doi_list) == len(retracted_doi_list):
        print(len(retracted_doi_list), 'retracted articles found.')
    else:
        print('Number of retraction articles and retracted articles are different: ',
              '{} vs. {}'.format(len(retractions_article_list), len(retracted_article_list)))
    return retractions_doi_list, retracted_doi_list

## Get list of corrected articles

In [2]:
article_list = [doi_to_file(doi) for doi in get_random_list_of_dois(count=100)]
corrections_article_list, corrected_article_list = get_corrected_article_list(article_list=article_list)

5 corrected articles found.


In [3]:
print(corrected_article_list)

['10.1371/journal.pone.0065474', '10.1371/journal.pone.0144760', '10.1371/journal.pone.0050818', '10.1371/journal.pmed.1001786', '10.1371/journal.ppat.1003068']


# Check raw XML for article updates

In [ ]:
# By default, checks only the 30,000 most recent articles
articles_different_list = revisiondate_sanity_check()
print(articles_different_list)

# DOI and filename sanity check

In [ ]:
# Check if article filenames match their full DOIs & that DOI fields are correct
messed_up_plos_list = article_doi_sanity_check()
messed_up_pmc_list = article_doi_sanity_check(directory=pmcdir, article_list=None, source='PMC')

# PubMed Corpus

## Get all local, solr, and PMC DOIs

In [3]:
plos_articles = get_all_plos_dois()
doi_to_pmc = get_articles_by_doi_field(check_new=False)
pmc_articles = list(doi_to_pmc.keys())

Articles that needs to be re-indexed on Solr:
10.1371/journal.pone.0076809


## Compare PLOS's copy to PMC

For sets of PLOS's corpus from PMC and PLOS, see which article are missing from PLOS's version of the corpus by:
* removing Currents articles
* checking if articles are live on journals.plos.org
* checking that the DOIs resolve

In [3]:
missing_plos_articles = process_missing_plos_articles(pmc_articles=pmc_articles, plos_articles=plos_articles)

## Compare PMC's copy to PLOS

For sets of PLOS's corpus from PMC and PLOS, see which article are missing from PMC's version of the Corpus by:
* updating the PMCID:DOI mapping document
* removing articles too recent to be indexed (pubdate less than 3 weeks ago)
* excluding uncorrected proofs
* excluding PLOS Medicine quizzes

In [4]:
missing_pmc_articles = process_missing_pmc_articles(pmc_articles=pmc_articles, plos_articles=plos_articles)

KeyboardInterrupt: 

## Save lists of missing articles to text files if needed

In [ ]:
with open('missing_plos_articles.txt', 'w') as file:
    for item in sorted(set(missing_plos_articles)):
        file.write("%s\n" % item)

In [8]:
id_list=listdir_nohidden(pmcdir, extension='.nxml')

In [9]:
doi_to_pmc = get_pmc_doi_dict(id_list)

KeyError: 'doi'

In [ ]:
with open('missing_pmc_articles.txt', 'w') as file:
    for item in sorted(set(missing_pmc_articles)):
        file.write("%s\n" % item)